In [153]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error as MSE
from glob import glob
import random
import pickle

In [127]:
data_files = glob("../../../zadanie2_7z/hmg/joined/*.csv")

In [114]:
#split train / val 85% / 15 %
random.seed(234)
val_split = round(len(data_files)*0.15)
val_files = random.sample(data_files, val_split)
train_files = [x for x in data_files if x not in val_files]

In [61]:
def split_seq(data_x, data_y, steps):
    x, y = list(), list()
    for i in range(len(data_x) - steps):
        end = i + steps
        seq_x = data_x[i:end, :]
        seq_y = data_y[end, :]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [131]:
dane

,001FCx00285_SPPV.PV,001XXXCALC01.NUM.PV[3],001SCx00274_SPPV.PV,001FCx00241_sppv.pv,001NIR0SZR0.daca.pv,001NIR0SZRG.daca.pv,001NIR0S600.daca.pv,001NIR0S500.daca.pv,001NIR0S300.daca.pv,001NIR0S100.daca.pv,...,001XXXCALC01.NUM.PV[2],prob_corg,prob_s,sita_nadziarno,sita_podziarno,poziom_zuzel,001UCx00274.pv,001NIR0ODS0.daca.pv,temp_zuz,007SxR00555.daca1.pv
0,2400,77.0,50.0,28.0,20.251532,0.782104,1.372570,5.047619,6.479706,6.590962,...,300.0,8.43,11.20,2.10,65.099998,1230,12.310957,10.477128,1309,-0.090600
1,2400,77.0,50.0,28.0,20.250053,0.782063,1.372521,5.047069,6.479192,6.590659,...,300.0,8.43,11.20,2.10,65.099998,1320,12.319610,10.477081,1315,0.101184
2,2400,77.0,50.0,28.0,20.248573,0.782023,1.372473,5.046519,6.478679,6.590356,...,300.0,8.43,11.20,2.10,65.099998,1320,12.345356,10.477036,1315,0.333862
3,2400,77.0,50.0,28.0,20.247093,0.781982,1.372424,5.045969,6.478166,6.590054,...,300.0,8.43,11.20,2.10,65.099998,1320,12.358917,10.476990,1315,0.338981
4,2400,77.0,50.0,28.0,20.245613,0.781942,1.372376,5.045419,6.477652,6.589750,...,300.0,8.43,11.20,2.10,65.099998,1320,12.360294,10.476943,1315,0.344100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86395,2700,79.0,44.0,27.0,25.355730,0.947532,1.710715,8.073397,7.722655,6.866261,...,306.0,8.88,11.08,2.32,60.599998,1270,14.431029,9.750987,1303,0.358161
86396,2700,79.0,44.0,27.0,25.355972,0.947545,1.710747,8.074211,7.722984,6.866515,...,306.0,8.88,11.08,2.32,60.599998,1270,14.431000,9.750924,1303,0.292953
86397,2700,79.0,44.0,27.0,25.356213,0.947558,1.710778,8.075027,7.723313,6.866770,...,306.0,8.88,11.08,2.32,60.599998,1270,14.446751,9.750862,1303,0.277618
86398,2700,79.0,44.0,27.0,25.356455,0.947571,1.710810,8.075842,7.723957,6.867024,...,306.0,8.88,11.08,2.32,60.599998,1270,14.478279,9.750799,1303,0.047500


In [130]:
dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
dane_x = dane.to_numpy()

In [ ]:
# exp 1 _ steps = 5

n_steps = 5
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp1/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp1/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 2 _ steps = 5, simpler model

n_steps = 5
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp2/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp2/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 1 _ steps = 10

n_steps = 10
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp3/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp3/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 4 _ steps = 30

n_steps = 30
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp4/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp4/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 5 _ steps = 60

n_steps = 60
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp5/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp5/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 6 _ steps = 30

n_steps = 30
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp6/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp6/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 7 _ steps = 20

n_steps = 30
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 500, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp7/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp7/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [75]:
dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()

dane_x = dane.to_numpy()

In [76]:
input_x, input_y = split_seq(dane_x, dane_y, steps = n_steps)

In [77]:
y_pred = model.predict(input_x, verbose=0)

In [80]:
MSE(input_y, y_pred)

3626.4536384230046

In [82]:
input_y[1]

array([2.33180008e+01, 8.70855033e-01, 1.51373994e+00, 7.21114254e+00,
       7.29695272e+00, 6.44669533e+00, 1.14000000e+02, 1.00000000e+01,
       3.00000000e+00, 3.49889832e+02, 3.12519609e+04, 9.90577539e+03,
       2.99000000e+02, 8.68000031e+00, 1.12200003e+01, 2.27999997e+00,
       6.30000000e+01, 1.37000000e+03, 1.36572247e+01, 9.36909103e+00,
       1.31000000e+03, 2.46942893e-01])

In [83]:
y_pred[1]

array([ 2.1546745e+01,  7.4828500e-01, -6.0043967e-01,  3.5830178e+00,
        5.5966163e+00,  1.3751291e+01,  1.1395613e+02,  1.1289093e+01,
        9.4705683e-01,  3.4296664e+02,  3.1261605e+04,  9.8865430e+03,
        2.9139093e+02,  1.1230208e+01,  8.0696602e+00,  6.9680538e+00,
        6.8245575e+01,  1.3641378e+03,  9.3218222e+00,  8.8075914e+00,
        1.2924592e+03,  4.1678696e+00], dtype=float32)